In [53]:
import numpy as np
import pandas as pd

# Collect data

### demographics

In [54]:
#user= dempgraphics?
users=pd.read_csv('demographics.csv')
users.columns=['Customer ID','Count','Gender','Age','Under 30','Senior Citizen','Married','Dependents','Number of Dependents']
# do we still need Under 30/ Senior Citizen, or we need to group the age ?
users.drop(['Count'],axis=1,inplace=True)

'''
#handle Gender NaN problem and turn into numeric
users.loc[users.Gender=='Male','Gender']='1'
users.loc[users.Gender=='Female','Gender']='2'
users.loc[users.Gender.isnull(),'Gender']='3'
users['Gender']=pd.to_numeric(users.Gender,errors='coerce')

#handle Married NaN problem and turn into numeric
users.loc[users.Married=='Yes','Married']='1'
users.loc[users.Married=='No','Married']='2'
users.loc[users.Married.isnull(),'Married']='3'
users['Married']=pd.to_numeric(users.Married,errors='coerce')
'''
users

,Customer ID,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents
0,1746-TGTWV,Male,78.0,No,Yes,No,No,0.0
1,8499-BRXTD,Female,74.0,No,Yes,Yes,NaN,1.0
2,2041-JIJCI,Male,71.0,No,Yes,No,Yes,3.0
3,5832-EXGTT,Female,78.0,No,Yes,Yes,Yes,1.0
4,8350-NYMVI,NaN,80.0,No,Yes,NaN,Yes,1.0
...,...,...,...,...,...,...,...,...
6158,7609-NRNCA,Female,NaN,No,No,No,No,0.0
6159,4835-YSJMR,Male,38.0,No,No,NaN,Yes,2.0
6160,5067-DGXLL,NaN,30.0,No,No,Yes,Yes,2.0
6161,3129-AAQOU,Female,32.0,No,No,Yes,Yes,2.0


### location

In [55]:
location=pd.read_csv('location.csv')
location.columns=['Customer ID','Count','Country','State','City','Zip Code','Lat Long','Latitude','Longtitude']
# Country, State are the same, latitude/longtitude's information is in Zip code
location.drop(['Count','Country','State','Lat Long','Latitude','Longtitude'],axis=1,inplace=True)

### population

In [56]:
population=pd.read_csv('population.csv')
population.columns=['ID','Zip Code','Population']
population.drop(['ID'],axis=1,inplace=True)
population

,Zip Code,Population
0,90001,54492
1,90002,44586
2,90003,58198
3,90004,67852
4,90005,43019
...,...,...
1666,96145,4002
1667,96146,942
1668,96148,678
1669,96150,33038


### merge location and population ,and join users

In [57]:
location=pd.merge(location,population,on='Zip Code')
users=pd.merge(users,location,on='Customer ID',how='outer')
users

,Customer ID,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,City,Zip Code,Population
0,1746-TGTWV,Male,78.0,No,Yes,No,No,0.0,Los Angeles,90022.0,68701.0
1,8499-BRXTD,Female,74.0,No,Yes,Yes,NaN,1.0,NaN,NaN,NaN
2,2041-JIJCI,Male,71.0,No,Yes,No,Yes,3.0,NaN,NaN,NaN
3,5832-EXGTT,Female,78.0,No,Yes,Yes,Yes,1.0,Inglewood,90303.0,27778.0
4,8350-NYMVI,NaN,80.0,No,Yes,NaN,Yes,1.0,Whittier,90602.0,26265.0
...,...,...,...,...,...,...,...,...,...,...,...
6853,6839-ITVZJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jacumba,91934.0,699.0
6854,6821-JPCDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glennville,93226.0,296.0
6855,5799-JRCZO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bakersfield,93311.0,20440.0
6856,8741-LQOBK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mccloud,96057.0,1586.0


satisfaction

In [58]:
satisfaction=pd.read_csv('satisfaction.csv')
satisfaction.columns=['Customer ID','score']
users=pd.merge(users,satisfaction,on='Customer ID',how='outer')

services

In [59]:
services=pd.read_csv('services.csv')
services.columns=['Customer ID','Count','Quarter','Referred a Friend','Number of Referrals','Tenure in Months','Offer','Phone Service','Avg Monthly Long Distance Charges','Multiple Lines','Internet Service','Internet Type','Avg Monthly GB Download','Online Security','Online Backup','Device Protection Plan','Premium Tech Support','Streaming TV','Streaming Movies','Streaming Music','Unlimited Data','Contract','Paperless Billing','Payment Method','Monthly Charge','Total Charges','Total Refunds','Total Extra Data Charges','Total Long Distance Charges','Total Revenue']
# Quarter are always Q3
services.drop(['Count','Quarter'],axis=1,inplace=True)
users=pd.merge(users,services,on='Customer ID',how='outer')
df=pd.DataFrame(users)
df["Married"]=pd.util.hash_array(df["Married"].to_numpy())
df["Gender"]=pd.util.hash_array(df["Gender"].to_numpy())
df["Under 30"]=pd.util.hash_array(df["Under 30"].to_numpy())
df["Senior Citizen"]=pd.util.hash_array(df["Senior Citizen"].to_numpy())
df["Dependents"]=pd.util.hash_array(df["Dependents"].to_numpy())
df["City"]=pd.util.hash_array(df["City"].to_numpy())
#df["Referred a friend"]=pd.util.hash_array(df["Referred a friend"].to_numpy())
df["Offer"]=pd.util.hash_array(df["Offer"].to_numpy())
df["Phone Service"]=pd.util.hash_array(df["Phone Service"].to_numpy())
df["Multiple Lines"]=pd.util.hash_array(df["Multiple Lines"].to_numpy())
df["Internet Service"]=pd.util.hash_array(df["Internet Service"].to_numpy())
df["Internet Type"]=pd.util.hash_array(df["Internet Type"].to_numpy())
df["Online Security"]=pd.util.hash_array(df["Online Security"].to_numpy())
df["Online Backup"]=pd.util.hash_array(df["Online Backup"].to_numpy())
df["Device Protection Plan"]=pd.util.hash_array(df["Device Protection Plan"].to_numpy())
df["Premium Tech Support"]=pd.util.hash_array(df["Premium Tech Support"].to_numpy())
df["Streaming TV"]=pd.util.hash_array(df["Streaming TV"].to_numpy())
df["Streaming Movies"]=pd.util.hash_array(df["Streaming Movies"].to_numpy())
df["Streaming Music"]=pd.util.hash_array(df["Streaming Music"].to_numpy())
df["Unlimited Data"]=pd.util.hash_array(df["Unlimited Data"].to_numpy())
df["Contract"]=pd.util.hash_array(df["Contract"].to_numpy())
df["Paperless Billing"]=pd.util.hash_array(df["Paperless Billing"].to_numpy())
df["Payment Method"]=pd.util.hash_array(df["Payment Method"].to_numpy())
df

,Customer ID,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,City,Zip Code,...,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue
0,1746-TGTWV,15160495269156175915,78.0,194712097405341480,6084295973400550908,194712097405341480,194712097405341480,0.0,14348480317110559103,90022.0,...,194712097405341480,10793207172922023642,6084295973400550908,149721189501102665,39.65,NaN,0.0,20.0,0.00,59.65
1,8499-BRXTD,11097954586364618109,74.0,194712097405341480,6084295973400550908,6084295973400550908,18446744073709551615,1.0,18446744073709551615,NaN,...,6084295973400550908,10793207172922023642,6084295973400550908,18446744073709551615,80.65,633.30,0.0,0.0,390.80,1024.10
2,2041-JIJCI,15160495269156175915,71.0,194712097405341480,6084295973400550908,194712097405341480,6084295973400550908,3.0,18446744073709551615,NaN,...,18446744073709551615,18446744073709551615,18446744073709551615,18446744073709551615,NaN,NaN,NaN,NaN,NaN,NaN
3,5832-EXGTT,11097954586364618109,78.0,194712097405341480,6084295973400550908,6084295973400550908,6084295973400550908,1.0,16801335329401301863,90303.0,...,6084295973400550908,10793207172922023642,6084295973400550908,149721189501102665,98.50,2514.50,NaN,0.0,494.00,2995.07
4,8350-NYMVI,18446744073709551615,80.0,194712097405341480,6084295973400550908,18446744073709551615,6084295973400550908,1.0,4904908573160186164,90602.0,...,6084295973400550908,10793207172922023642,6084295973400550908,149721189501102665,76.50,2868.15,0.0,0.0,234.21,3102.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,0516-WJVXC,18446744073709551615,NaN,18446744073709551615,18446744073709551615,18446744073709551615,18446744073709551615,NaN,18446744073709551615,NaN,...,18446744073709551615,10793207172922023642,194712097405341480,7072025398692509137,19.70,NaN,0.0,0.0,NaN,NaN
7038,0388-EOPEX,18446744073709551615,NaN,18446744073709551615,18446744073709551615,18446744073709551615,18446744073709551615,NaN,18446744073709551615,NaN,...,194712097405341480,10748406164826253654,194712097405341480,7072025398692509137,20.75,700.45,0.0,0.0,1242.50,1942.95
7039,7055-JCGNI,18446744073709551615,NaN,18446744073709551615,18446744073709551615,18446744073709551615,18446744073709551615,NaN,18446744073709551615,NaN,...,6084295973400550908,16268543566089291274,194712097405341480,7072025398692509137,NaN,6631.85,0.0,0.0,1803.75,8435.60
7040,6891-JPYFF,18446744073709551615,NaN,18446744073709551615,18446744073709551615,18446744073709551615,18446744073709551615,NaN,18446744073709551615,NaN,...,6084295973400550908,10793207172922023642,6084295973400550908,7072025398692509137,50.70,50.70,NaN,0.0,38.05,88.75


status

In [60]:
status=pd.read_csv('status.csv')
status.columns=['Customer ID','Churn Category']

#final train data
train=pd.merge(status,df,on='Customer ID',how='left')

# Train Model

In [61]:
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 

In [62]:
# give up every string, since there are NaN
features=list(train)
features.remove('Customer ID')
features.remove('Churn Category')
features.remove('Referred a Friend')
#features.remove('Offer')
'''
features.remove('Under 30')
features.remove('Married')
features.remove('Senior Citizen')
features.remove('Dependents')
features.remove('City')
features.remove('Referred a Friend')
features.remove('Offer')
features.remove('Phone Service')
features.remove('Multiple Lines')
features.remove('Internet Service')
features.remove('Internet Type')
features.remove('Online Security')
features.remove('Online Backup')
features.remove('Device Protection Plan')
features.remove('Premium Tech Support')
features.remove('Streaming TV')
features.remove('Streaming Movies')
features.remove('Streaming Music')
features.remove('Unlimited Data')
features.remove('Contract')
features.remove('Paperless Billing')
features.remove('Payment Method')
'''
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer = imputer.fit(train.loc[:, features])
train_imputed = imputer.fit_transform(train.loc[:, features])

In [63]:
# 10% vaildation 
x_train,x_test,y_train,y_test = train_test_split(train_imputed,train.loc[:, 'Churn Category'],test_size=0.90, random_state=0)

### SVM_linear 

In [64]:

svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_train, y_train)
svm_predictions = svm_model_linear.predict(x_test)
accuracy = svm_model_linear.score(x_test, y_test)
print(accuracy)

0.02917981072555205


### decision tree

In [65]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 1).fit(x_train, y_train)
dtree_predictions = dtree_model.predict(x_test)
accuracy = dtree_model.score(x_test, y_test)
print(accuracy)

0.8091482649842271


### knn 

In [66]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 38).fit(x_train, y_train)
accuracy = knn.score(x_test, y_test)
print(accuracy)


0.7389589905362776


### RandomForest + ada boost

In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

rf = RandomForestClassifier(n_estimators = 1000, random_state = 0,n_jobs=-1)
ada = AdaBoostClassifier(n_estimators = 1000, random_state = 0, )



rf.fit(x_train, y_train)
rf.score(x_train, y_train)

#ada.fit(x_train, y_train)
#ada.score(x_train, y_train)

1.0

In [68]:
rf.score(x_test, y_test)

0.8101997896950578

### Prediction

In [69]:
testID=pd.read_csv('Test_IDs.csv')
testID.columns=['Customer ID']
dftest=pd.DataFrame(testID)
test=pd.merge(dftest,df,on='Customer ID',how='left')
#print(test)
test_imputed = imputer.fit_transform(test.loc[:, features])
print(test_imputed)
#dftest['Churn Category']=svm_model_linear.predict(test_imputed)

#dftest['Churn Category']=dtree_model.predict(test_imputed)

#testID['Churn Category']=knn.predict(test_imputed)

#dftest['Churn Category']=rfr.predict(test_imputed)
dftest['Churn Category']=rf.predict(test_imputed)
#dftest['Churn Category']=ada.predict(test_imputed)

[[1.10979546e+19 7.30000000e+01 1.94712097e+17 ... 0.00000000e+00
  1.26665000e+03 4.54891000e+03]
 [1.51604953e+19 6.40000000e+01 1.94712097e+17 ... 0.00000000e+00
  2.35800000e+03 3.55325000e+03]
 [1.84467441e+19 4.70000000e+01 1.84467441e+19 ... 0.00000000e+00
  5.40800000e+02 4.40625000e+03]
 ...
 [1.51604953e+19 2.70000000e+01 6.08429597e+18 ... 0.00000000e+00
  4.58430000e+02 2.13289000e+03]
 [1.10979546e+19 3.80000000e+01 1.84467441e+19 ... 0.00000000e+00
  4.07110000e+02 6.40780000e+02]
 [1.84467441e+19 4.70000000e+01 1.84467441e+19 ... 0.00000000e+00
  1.71300000e+02 2.85150000e+02]]


### Output result

In [70]:
testID.columns=['Customer ID','Churn Category']
testID['ans'] = testID['Churn Category']
testID.loc[testID.ans=='No Churn','ans']='0'
testID.loc[testID.ans=='Competitor','ans']='1'
testID.loc[testID.ans=='Dissatisfaction','ans']='2'
testID.loc[testID.ans=='Attitude','ans']='3'
testID.loc[testID.ans=='Price','ans']='4'
testID.loc[testID.ans=='Other','ans']='5'
testID.drop(['Churn Category'],axis=1,inplace=True)
testID.rename(columns={'ans':'Churn Category'}, inplace=True)
submiss=pd.DataFrame(testID)
submiss.to_csv('submission.csv',index=False)